# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data')
print(song_files[0])

/home/workspace/data/song_data/A/A/A/TRAAAAW128F429D538.json


In [5]:
filepath = song_files[0]
print(filepath)

/home/workspace/data/song_data/A/A/A/TRAAAAW128F429D538.json


In [6]:
try:
    df = pd.read_json(filepath, typ='series')
    #df = pd.read_json(filepath, lines=True)
except Exception as e:
    print("Read json error")
    print(e)
df.head(20)

num_songs                            1
artist_id           ARD7TVE1187B99BFB1
artist_latitude                   None
artist_longitude                  None
artist_location        California - LA
artist_name                     Casual
song_id             SOMZWCG12A8C13C480
title                 I Didn't Mean To
duration                       218.932
year                                 0
dtype: object

In [7]:
try:
    cur.execute("CREATE TABLE IF NOT EXISTS test_table (artist_name varchar, song_id varchar, title varchar);")
    cur.execute("INSERT INTO test_table (artist_name, song_id, title) VALUES (%s, %s, %s)", ("test", "test", "test"))
    cur.execute("INSERT INTO test_table (artist_name, song_id, title) VALUES (%s, %s, %s)", ("chung", "pham", "test"))
except Exception as e:
    print("error ", e)



In [8]:
test_data = (df['artist_name'], df['song_id'], df['title'])
#print(test_data)
print(type(test_data))
#song_table_insert = ("""
#INSERT INTO songs (song_id, title, artist_id, year, duration)
              #  VALUES (%s, %s, %s, %s, %s)""" 
cur.execute('INSERT INTO test_table (artist_name, song_id, title) VALUES (%s,%s,%s)', test_data)

<class 'tuple'>


In [9]:
cur.execute("SELECT * FROM test_table;")
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

('test', 'test', 'test')
('chung', 'pham', 'test')
('Casual', 'SOMZWCG12A8C13C480', "I Didn't Mean To")


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [10]:
song_data = df.filter(items=['song_id', 'title', 'artist_id', 'year', 'duration']).tolist()
#song_data = df.filter(items=['song_id', 'title', 'artist_id', 'year', 'duration']).values[0].tolist()
print(type(song_data))

<class 'list'>


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [11]:
cur.execute(song_table_insert, song_data)
conn.commit()


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [12]:
artist_data = df.filter(items=['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']).tolist()
#artist_data

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [13]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [14]:
log_files = get_files('data/log_data')

In [15]:
filepath = log_files[0]

In [16]:
try:
    #df = pd.read_json(filepath, typ='series')
    df = pd.read_json(filepath, lines=True)
except Exception as e:
    print("error: ", e)
df.head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [17]:
df = pd.read_json(filepath, lines=True)

In [18]:
#df['ts'].head()
try:
    t = pd.to_datetime(df['ts'], unit='ms')
except Exception as e:
    print(e)

In [19]:
try:
    time_data = [t, t.dt.hour, t.dt.day, t.dt.week, t.dt.month, t.dt.year, t.dt.dayofweek]

except Exception as e:
    print(e)
#print(time_data)
column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')

In [20]:
try:
    time_df = pd.DataFrame(dict(zip(column_labels, time_data)))
except Exception as e:
    print(e)
#time_df.head()

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [22]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

InternalError: current transaction is aborted, commands ignored until end of transaction block


Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [23]:
user_columns = ('user_id', 'first_name', 'last_name', 'gender', 'level')
user_data = [df['userId'], df['firstName'], df['lastName'], df['gender'], df['level']]
try:
    user_df = pd.DataFrame(dict(zip(user_columns, user_data)))
except Exception as e:
    print(e)

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [24]:
count = 0
count2 = 0;
for i, row in user_df.iterrows():
    count2 = i
    try:
        cur.execute(user_table_insert, row)
        conn.commit()
    except Exception as e:
        count += 1
print(count)
print(count2)

388
387


Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [25]:
song_select_1 = ("""
SELECT song_id, songs.artist_id, duration FROM songs JOIN artists ON artists.artist_id=songs.artist_id;
""")
try:
    cur.execute(song_select_1)
except Exception as e:
    print(e)


current transaction is aborted, commands ignored until end of transaction block



In [26]:
for index, row in df.iterrows():
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    #songplay_id, start_time, user_id, level, song_id, artist_id, session_id, location, user_agent
    songplay_data = (0, row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

InternalError: current transaction is aborted, commands ignored until end of transaction block


Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [23]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.